In [75]:
%%bash

set -e

TARGET_DIRECTORY=duquantum/
if [[ -d $TARGET_DIRECTORY ]]; then
    echo "Deleting $TARGET_DIRECTORY..."
    rm -rf $TARGET_DIRECTORY
fi

echo "-----------------------------------"
REPO=https://github.com/Dpbm/duquantum
echo "Cloning repo from $REPO ..."
git clone $REPO $TARGET_DIRECTORY
echo "Directory ($TARGET_DIRECTORY) data:"
ls -la $TARGET_DIRECTORY

echo "-----------------------------------"
PYTHON_VERSION=$(cat $TARGET_DIRECTORY/.python-version)
echo "You must install python version: $PYTHON_VERSION"
python_dependencies=("python$PYTHON_VERSION" "python$PYTHON_VERSION-venv")
echo "Required dependencies: ${python_dependencies[@]}"
echo "Installing apt dependencies..." && \
    sudo apt update && \
    sudo apt install -y "${python_dependencies[@]}"

Deleting duquantum/...
-----------------------------------
Cloning repo from https://github.com/Dpbm/duquantum ...
Directory (duquantum/) data:
total 64
drwxr-xr-x 3 root root  4096 Jan 31 17:37 .
drwxr-xr-x 1 root root  4096 Jan 31 17:37 ..
-rw-r--r-- 1 root root   402 Jan 31 17:37 CMakeLists.txt
-rw-r--r-- 1 root root    63 Jan 31 17:37 conanfile.txt
-rw-r--r-- 1 root root   251 Jan 31 17:37 dev-requirements.txt
drwxr-xr-x 8 root root  4096 Jan 31 17:37 .git
-rw-r--r-- 1 root root  6129 Jan 31 17:37 .gitignore
-rw-r--r-- 1 root root 10280 Jan 31 17:37 LICENSE
-rw-r--r-- 1 root root   802 Jan 31 17:37 Makefile
-rw-r--r-- 1 root root     5 Jan 31 17:37 .python-version
-rw-r--r-- 1 root root     0 Jan 31 17:37 README.md
-rw-r--r-- 1 root root  1047 Jan 31 17:37 statevec.cu
-rw-r--r-- 1 root root  6223 Jan 31 17:37 statevec.hpp
-----------------------------------
You must install python version: 3.12
Required dependencies: python3.12 python3.12-venv
Installing apt dependencies...
Hit:1 h

Cloning into 'duquantum'...


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




In [76]:
!nvidia-smi

Sat Jan 31 17:37:58 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [85]:
%%bash

set -e

CUDA_VERSION=$(nvcc --version | tail -n 1 | grep -Eo 'cuda_[0-9]{2}' | sed 's|cuda_||g')
echo "Using cuda ${CUDA_VERSION}!"

cd duquantum
echo "DuQuantum directory ($(pwd)):"
ls -la

python3.13 -m venv .venv
source .venv/bin/activate

echo "------------------------------"
echo "Installing dependencies..."
pip install -r dev-requirements.txt
pip install "cuquantum-cu${CUDA_VERSION}"


Using cuda 12!
DuQuantum directory (/content/duquantum):
total 68
drwxr-xr-x 4 root root  4096 Jan 31 17:41 .
drwxr-xr-x 1 root root  4096 Jan 31 17:37 ..
-rw-r--r-- 1 root root   402 Jan 31 17:37 CMakeLists.txt
-rw-r--r-- 1 root root    63 Jan 31 17:37 conanfile.txt
-rw-r--r-- 1 root root   251 Jan 31 17:37 dev-requirements.txt
drwxr-xr-x 8 root root  4096 Jan 31 17:37 .git
-rw-r--r-- 1 root root  6129 Jan 31 17:37 .gitignore
-rw-r--r-- 1 root root 10280 Jan 31 17:37 LICENSE
-rw-r--r-- 1 root root   802 Jan 31 17:37 Makefile
-rw-r--r-- 1 root root     5 Jan 31 17:37 .python-version
-rw-r--r-- 1 root root     0 Jan 31 17:37 README.md
-rw-r--r-- 1 root root  1047 Jan 31 17:37 statevec.cu
-rw-r--r-- 1 root root  6223 Jan 31 17:37 statevec.hpp
drwxr-xr-x 5 root root  4096 Jan 31 17:41 .venv
------------------------------
Installing dependencies...


In [101]:
%%bash

set -e

echo "Installing dev dependencies..."
sudo apt update && sudo apt install -y build-essential cmake make binutils clang ninja-build

echo "building project and running tests..."
cd duquantum
echo "DuQuantum directory ($(pwd)):"
ls -la

source .venv/bin/activate

# make setup-conan
conan profile detect --force

#PYTHON_VERSION=$(cat .python-version)
PYTHON_VERSION=3.13
CUQUANTUM_PATH="$(pwd)/.venv/lib/python${PYTHON_VERSION}/site-packages/cuquantum"

CUQUANTUM_INCLUDE_DIRS="$CUQUANTUM_PATH/include"
CUQUANTUM_LIB_DIRS="$CUQUANTUM_PATH/lib"
CUDA_INCLUDE_DIRS="/usr/local/cuda/include"

echo "USING: "
echo "PYTHON VERSION: $PYTHON_VERSION"
echo "CUQUANTUM PATH: $CUQUANTUM_PATH"
echo "CUQUANTUM INCLUDE DIRS: $CUQUANTUM_INCLUDE_DIRS"
echo "CUQUANTUM LIB DIRS: $CUQUANTUM_LIB_DIRS"
echo "CUDA INCLUDE DIRS: $CUDA_INCLUDE_DIRS"

make delete && \
    make CUDA_INCLUDE_DIRS=$CUDA_INCLUDE_DIRS CUQUANTUM_INCLUDE_DIRS=$CUQUANTUM_INCLUDE_DIRS CUQUANTUM_LIB_DIRS=$CUQUANTUM_LIB_DIRS && \
    cd ./build/tests && \
    ctest
    #make run-tests


Installing dev dependencies...
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,544 B/s)
Reading package lists...
Building dependency tree...
Reading state information...
96 packages can be upgraded. Run 'apt list --upgradable' to see them.
Readin



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
detect_api: Found cc=gcc-11.4.0
detect_api: gcc>=5, using the major as version
detect_api: gcc C++ standard library: libstdc++11

Detected profile:
WARN: This profile is a guess of your environment, please check it.
WARN: The output of this command is not guaranteed to be stable and can change in future Conan versions.
WARN: Use your own profile files for 

CalledProcessError: Command 'b'\nset -e\n\necho "Installing dev dependencies..."\nsudo apt update && sudo apt install -y build-essential cmake make binutils clang ninja-build\n\necho "building project and running tests..."\ncd duquantum\necho "DuQuantum directory ($(pwd)):"\nls -la\n\nsource .venv/bin/activate\n\n# make setup-conan\nconan profile detect --force\n\n#PYTHON_VERSION=$(cat .python-version)\nPYTHON_VERSION=3.13\nCUQUANTUM_PATH="$(pwd)/.venv/lib/python${PYTHON_VERSION}/site-packages/cuquantum"\n\nCUQUANTUM_INCLUDE_DIRS="$CUQUANTUM_PATH/include"\nCUQUANTUM_LIB_DIRS="$CUQUANTUM_PATH/lib"\nCUDA_INCLUDE_DIRS="/usr/local/cuda/include"\n\necho "USING: "\necho "PYTHON VERSION: $PYTHON_VERSION"\necho "CUQUANTUM PATH: $CUQUANTUM_PATH"\necho "CUQUANTUM INCLUDE DIRS: $CUQUANTUM_INCLUDE_DIRS"\necho "CUQUANTUM LIB DIRS: $CUQUANTUM_LIB_DIRS"\necho "CUDA INCLUDE DIRS: $CUDA_INCLUDE_DIRS"\n\nmake delete && \\\n    make CUDA_INCLUDE_DIRS=$CUDA_INCLUDE_DIRS CUQUANTUM_INCLUDE_DIRS=$CUQUANTUM_I

In [ ]:
! ls -la 

total 56
drwxr-xr-x 1 root root 4096 Jul 10  2024 .
drwxr-xr-x 1 root root 4096 Jan 29 14:22 ..
drwxr-xr-x 3 root root 4096 Jul 10  2024 bin
drwxr-xr-x 2 root root 4096 Jul 10  2024 compat
drwxr-xr-x 5 root root 4096 Jul 10  2024 compute-sanitizer
drwxr-xr-x 3 root root 4096 Jul 10  2024 doc
drwxr-xr-x 4 root root 4096 Jul 10  2024 extras
drwxr-xr-x 2 root root 4096 Jul 10  2024 gds
lrwxrwxrwx 1 root root   28 Jun  6  2024 include -> targets/x86_64-linux/include
lrwxrwxrwx 1 root root   24 Jun  6  2024 lib64 -> targets/x86_64-linux/lib
drwxr-xr-x 3 root root 4096 Jul 10  2024 nvml
drwxr-xr-x 6 root root 4096 Jul 10  2024 nvvm
drwxr-xr-x 3 root root 4096 Jul 10  2024 share
drwxr-xr-x 2 root root 4096 Jul 10  2024 src
drwxr-xr-x 1 root root 4096 Jul 10  2024 targets
